In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.models.rnn import rnn_cell
from tensorflow.models.rnn import rnn

from functools import partial

In [19]:
def slidingWindow(sequence,winSize,step=1):
    """Returns a generator that will iterate through
    the defined chunks of input sequence.  Input sequence
    must be iterable."""
 
    # Verify the inputs
    try: it = iter(sequence)
    except TypeError:
        raise Exception("**ERROR** sequence must be iterable.")
    if not ((type(winSize) == type(0)) and (type(step) == type(0))):
        raise Exception("**ERROR** type(winSize) and type(step) must be int.")
    if step > winSize:
        raise Exception("**ERROR** step must not be larger than winSize.")
    if winSize > len(sequence):
        raise Exception("**ERROR** winSize must not be larger than sequence length.")
 
    # Pre-compute number of chunks to emit
    numOfChunks = ((len(sequence)-winSize)/step)+1
 
    # Do the work
    for i in range(0,numOfChunks*step,step):
        yield sequence[i:i+winSize]
        
        
def adjust_sequence_to_length(sequence, length, meta, padder):        
    if len(sequence) > length:
        slices_num = float(len(sequence))/length
        step = max(int((slices_num%1)*length),2)
        chunks = list(slidingWindow(sequence, winSize=length, step=step))
        return chunks
    if len(sequence) == length:
        return [sequence]
    if len(sequence) < length:
        gap = length - len(sequence)        
        seq_pred = padder(sequence, gap, meta)            
        return [seq_pred]

In [28]:
class BatchGenerator(object):
    
    def __init__(self, batch_size):
        self._journeys_types = np.array([
#             [1,2,3,4,5,6,7],
#             [1,2,3,4,5],
#             [7,6,5,4,3,2,1],
#             [7,6,5,4,5],
#             [4,5,2,3,1,7,6],
#             [1,1,3,4,4,5,7],
#             [1,1,3,4,4],
#             [1,1,1,4,4,5,5],
#             [5,5,5,2,2,2,3,3,3]
                
                
            ([1,2,3, 7,6,5,4],[0]),
            ([1,2,3, 4,5,6,7],[1]),
            ([7,6,5, 4,3,2,1],[0]),
            ([7,6,5, 1,2,3,4],[1])
            
#             [1,2,3,4,5],
#             [7,6,5,4,3,2,1],
#             [7,6,5,4,5],
#             [4,5,2,3,1,7,6],
#             [1,1,3,4,4,5,7],
#             [1,1,3,4,4],
#             [1,1,1,4,4,5,5],
#             [5,5,5,2,2,2,3,3,3]
        ])
        self._batch_size = batch_size 
#         self._meta_vocabulary_size = meta_vocabulary_size
        
        journeys_types = map(lambda j: j[0], self._journeys_types)
        self._journey_vocabulary_size=len(np.unique([item for sublist in journeys_types for item in sublist]))
        
        meta_types = map(lambda j: j[1], self._journeys_types)
        self._meta_vocabulary_size=len(np.unique([item for sublist in meta_types for item in sublist]))
        
        self._vocabulary_size = self._journey_vocabulary_size + self._meta_vocabulary_size
    
    
    def val2id(self, val):
        return val-1
    
    
    def id2val(self, id):
        return id+1
    
    
    def encode_meta_1h(self, id):
        z = np.zeros(self._meta_vocabulary_size, dtype=np.float32)        
        z[id] = 1.0        
        return z.reshape((1, self._meta_vocabulary_size)) 
    
#     def batch2journeys(self, batch):
#         r = []
#         for i in zip(*batch):
#             ids = np.argmax(i, axis=1)                
#             r.append(map(self.id2val, ids))
#         return r
       
        
    def next_batch(self, unrollings_adjuster=None, num_unrollings=None):
        batch = []
        while len(batch)<self._batch_size:
            jrn_md = self._journeys_types[np.random.randint(0, len(self._journeys_types))]  
            jrn = jrn_md[0]
            meta_data = jrn_md[1]
            if unrollings_adjuster and num_unrollings:
                pb = unrollings_adjuster(jrn, num_unrollings, meta_data)               
                pb_md = map(lambda j: (j,meta_data),pb)                
                batch.extend(pb_md)      
            else:
                batch.append((jrn, meta_data))
        return map(lambda i: batch[i], np.random.choice(len(batch), self._batch_size, replace=False))
               
        
    def encode_batches(self, batches, num_unrollings):   
        
        jrns = map(lambda b:b[0], batches)
        metas = map(lambda b:b[1], batches)
                
        b_transposed = np.transpose(jrns)
        
        res = np.zeros((num_unrollings, self._batch_size, self._vocabulary_size), dtype=np.float32)
        for (unrolling, batch), value in np.ndenumerate(b_transposed):
            res[unrolling, batch, self.val2id(value)] = 1.0   
            meta_1h = self.encode_meta_1h(metas[batch])
#             print self._vocabulary_size-self._meta_vocabulary_size, self._vocabulary_size,'=>',meta
            res[unrolling, batch].put([self._vocabulary_size-self._meta_vocabulary_size, self._vocabulary_size-1], meta_1h)
        return res

    
    def encode_1h(self, jrn_step_id, meta_id=None):
        z = np.zeros(self._vocabulary_size, dtype=np.float32)        
        z[jrn_step_id] = 1.0        
        
        if meta_id:
            meta_1h = self.encode_meta_1h(meta_id)
            z.put([self._vocabulary_size-self._meta_vocabulary_size, self._vocabulary_size-1], meta_1h)
        
        return z.reshape((1, self._vocabulary_size))        
            

    def weighted_pick(self, weights):
        assert len(weights) == self._vocabulary_size - self._meta_vocabulary_size
        t = np.cumsum(weights)
        s = np.sum(weights)
        return(int(np.searchsorted(t, np.random.rand(1)*s)))

In [4]:
batch_size = 5
jrn_voc = 7
meta_voc = 2
vocabulary_size = jrn_voc+meta_voc
num_unrollings = 3
data_gen = BatchGenerator(batch_size)

batches = data_gen.next_batch(unrollings_adjuster=unrollings_adjuster, num_unrollings=3)
batches


NameError: name 'unrollings_adjuster' is not defined

In [ ]:
data_gen.encode_batches(batches, num_unrollings)

In [ ]:
batch = map(lambda b:b[0], batches)
batch

In [ ]:
meta = map(lambda b:b[1], batches)
meta

In [ ]:
b_transposed = np.transpose(batch)
b_transposed

In [ ]:
res = np.zeros((num_unrollings, batch_size, vocabulary_size), dtype=np.float32)
res

In [ ]:
for (unrolling, batch), value in np.ndenumerate(b_transposed):
    print unrolling, batch, value, meta[batch]
#     res[unrolling, batch, value-1] = 1.0 
    res[unrolling, batch].put([jrn_voc,jrn_voc+meta_voc-1], meta[batch])
#     res[unrolling, batch, self.val2id(value)] = 1.0   

In [ ]:
res

In [ ]:
res[0,0].put([jrn_voc,jrn_voc+meta_voc-1],[1,1])
res

In [ ]:
a = np.array([[1,2,3,4],[1,2,3,4]])
a[1,2] = 99
# a.put([0,[2,3]],np.array([5,6]))
a



In [29]:
# Parameters
n_hidden = 14 # hidden layer num of features
n_unrollings = 6 #10 #max journeys length => RNN unrolled length
batch_size = 5

data_gen = BatchGenerator(batch_size)

meta_vocabulary_size = data_gen._meta_vocabulary_size
total_vocabulary_size = data_gen._vocabulary_size

vocabulary_size = total_vocabulary_size - meta_vocabulary_size #total possible journey states


In [30]:
graph = tf.Graph()
with graph.as_default():        

    # Input data.
    train_data = list()    
    for _ in range(n_unrollings+1):
        train_data.append(tf.placeholder(tf.float32, shape=[batch_size, total_vocabulary_size]))        
        
    train_inputs = train_data[:n_unrollings]
    train_labels = train_data[1:]  # labels are inputs shifted by one time step.
    
    train_labels_concat_ = tf.concat(0, train_labels)
    s = train_labels_concat_.get_shape()     
    train_labels_concat = tf.slice(train_labels_concat_,  [0, 0], np.array(s.as_list())-np.array([0,meta_vocabulary_size]))
    
    
    
    #social Input data
#     social_input = tf.placeholder(tf.float32, shape=[batch_size, social_vocabulary_size])
    
    with tf.variable_scope("rnn") as rnn_scope:
    
        
    
        # RNN struct
        cell = rnn_cell.LSTMCell(num_units=n_hidden, input_size=total_vocabulary_size)     
        outputs, states = rnn.rnn(cell, train_inputs, dtype=tf.float32) 
        outputs_concat = tf.concat(0, outputs)#tf.reshape(tf.concat(1, outputs), (-1, n_hidden))

        # Classifier.
        W_hy = tf.get_variable("W_hy", [n_hidden, vocabulary_size])
        b_hy = tf.get_variable("b_hy", [vocabulary_size])
        
        
#         W_social = tf.get_variable("W_social", [n_hidden, social_vocabulary_size])
#         b_social = tf.get_variable("b_social", [social_vocabulary_size])
        
        logits = tf.matmul(outputs_concat, W_hy) + b_hy

        # Loss func
        loss = tf.reduce_mean(
                tf.nn.softmax_cross_entropy_with_logits(logits, train_labels_concat))

        
        # Optimizer.
        global_step = tf.Variable(0, name='global_step')
        learning_rate = tf.train.exponential_decay(10.0, global_step, 5000, 0.1, staircase=True)
        optimizer = tf.train.GradientDescentOptimizer(learning_rate)
        gradients, v = zip(*optimizer.compute_gradients(loss))
        gradients, _ = tf.clip_by_global_norm(gradients, 1.25)
        optimizer = optimizer.apply_gradients(
                zip(gradients, v), global_step=global_step)


        # Predictions.
        train_prediction = tf.nn.softmax(logits)
        
        #perplexity
#         predictions[predictions < 1e-10] = 1e-10
#         return np.sum(np.multiply(labels, -np.log(predictions))) / labels.shape[0]        
        perplexity = tf.reduce_sum(tf.mul(train_labels_concat, -tf.log(train_prediction)))/train_labels_concat.get_shape()[0].value
        
        loss_summary = tf.scalar_summary("loss", loss)
        learning_rate_summary = tf.scalar_summary("learning_rate", learning_rate)
        perplexity_summary = tf.scalar_summary("perplexity_summary", perplexity)
    


In [31]:
train_inputs

[<tf.Tensor 'Placeholder:0' shape=(5, 9) dtype=float32>,
 <tf.Tensor 'Placeholder_1:0' shape=(5, 9) dtype=float32>,
 <tf.Tensor 'Placeholder_2:0' shape=(5, 9) dtype=float32>,
 <tf.Tensor 'Placeholder_3:0' shape=(5, 9) dtype=float32>,
 <tf.Tensor 'Placeholder_4:0' shape=(5, 9) dtype=float32>,
 <tf.Tensor 'Placeholder_5:0' shape=(5, 9) dtype=float32>]

In [32]:
print outputs_concat.get_shape(), W_hy.get_shape(), '+',b_hy.get_shape(),'=>', logits.get_shape()

(30, 14) (14, 7) + (7,) => (30, 7)


In [33]:
print train_prediction.get_shape()

(30, 7)


In [34]:
outputs

[<tf.Tensor 'rnn/RNN/LSTMCell/mul_2:0' shape=(5, 14) dtype=float32>,
 <tf.Tensor 'rnn/RNN/LSTMCell_1/mul_2:0' shape=(5, 14) dtype=float32>,
 <tf.Tensor 'rnn/RNN/LSTMCell_2/mul_2:0' shape=(5, 14) dtype=float32>,
 <tf.Tensor 'rnn/RNN/LSTMCell_3/mul_2:0' shape=(5, 14) dtype=float32>,
 <tf.Tensor 'rnn/RNN/LSTMCell_4/mul_2:0' shape=(5, 14) dtype=float32>,
 <tf.Tensor 'rnn/RNN/LSTMCell_5/mul_2:0' shape=(5, 14) dtype=float32>]

In [35]:
train_labels_concat_,train_labels_concat

(<tf.Tensor 'concat:0' shape=(30, 9) dtype=float32>,
 <tf.Tensor 'Slice:0' shape=(30, 7) dtype=float32>)

In [36]:
predictors = {}
def get_predictor(inputs_number):
    
    if inputs_number in predictors:       
        return predictors[inputs_number]
    else:
    
        with graph.as_default():  

    #         rnn_scope.reuse_variables()
            with tf.variable_scope("rnn",reuse=True) as rnn_scope:

                sample_inputs = list()
                for _ in range(inputs_number):
                    sample_inputs.append(
                        tf.placeholder(tf.float32, shape=[None, total_vocabulary_size]))

                sample_outputs, sample_states = rnn.rnn(cell, sample_inputs ,dtype=tf.float32) 


                sample_output = tf.concat(0, sample_outputs)

                sample_logits = tf.matmul(sample_output, W_hy) + b_hy
                sample_prediction = tf.nn.softmax(sample_logits)

                predictors[inputs_number] = (sample_inputs, sample_prediction)
                return sample_inputs, sample_prediction

In [37]:
def predict(ini_steps, num_steps_forward, meta_id):
    
    jrn_ids = map(data_gen.val2id, ini_steps)
    encode_jrn_1h = partial(data_gen.encode_1h, meta_id=meta_id)
    jrn_1h = map(encode_jrn_1h, jrn_ids)
    
    total_steps = num_steps_forward + len(jrn_ids)
    for _ in range(total_steps-len(jrn_ids)):
        sample_inputs, model = get_predictor(len(jrn_ids))

        dick ={}
        for i, s in zip(sample_inputs, jrn_1h):                        
            dick[i]=s

        prediction = model.eval(dick)

        #take only last step
        last_pred_step = prediction[-1].ravel()
        pred_step_id = data_gen.weighted_pick(last_pred_step)                                          
        jrn_ids.append(pred_step_id)                 

        pred_step_1h = data_gen.encode_1h(pred_step_id, meta_id=meta_id)                    
        jrn_1h.append(pred_step_1h) 
        
    return map(lambda w: data_gen.id2val(w), jrn_ids)

In [38]:
num_epochs = 10000
summary_frequency = 100

unrollings_adjuster = partial(adjust_sequence_to_length, padder=predict)



# with tf.Session(graph=graph) as sess:
sess = tf.InteractiveSession(graph=graph)


merged_summaries = tf.merge_all_summaries()
writer = tf.train.SummaryWriter('/tmp/tensorflow_logs', sess.graph_def)

tf.initialize_all_variables().run()
# saver = tf.train.Saver(tf.all_variables())



print('Initialized')
mean_loss = 0
for step in range(num_epochs):

    batches = data_gen.next_batch(unrollings_adjuster=unrollings_adjuster, num_unrollings=n_unrollings+1)
    batches_encoded = data_gen.encode_batches(batches,n_unrollings+1)
    feed_dict = dict()
    # fill data into input placeholders
    for i in range(n_unrollings+1):
        feed_dict[train_data[i]] = batches_encoded[i]


    _, l, prplx, predictions, lr, summaries_str = sess.run([optimizer, loss, perplexity, train_prediction, learning_rate, merged_summaries], feed_dict=feed_dict)
    mean_loss += l

    
    
    if step % summary_frequency == 0:
        if step > 0:
            mean_loss = mean_loss / summary_frequency
        # The mean loss is an estimate of the loss over the last few batches.
        
        mean_loss = 0
        
        
#         labels = np.concatenate(list(batches_encoded)[1:])
#         perplexity = np.exp(logprob(predictions, labels))
        
        
        if step % summary_frequency == 0:
            print('Average loss at step %d: %f learning rate: %f' % (step, mean_loss, lr))
            print('Minibatch perplexity: %.2f' % float(prplx))
            writer.add_summary(summaries_str, step)
#             writer.add_summary(perplexity_summary, step)

        
        

#         pr = train_prediction.eval(feed_dict)
        pr = predictions
        print map(lambda w: data_gen.id2val(data_gen.weighted_pick(w)), pr)



Initialized
Average loss at step 0: 0.000000 learning rate: 10.000000
Minibatch perplexity: 2.25
[6, 6, 6, 4, 6, 6, 6, 6, 6, 4, 6, 6, 3, 5, 6, 3, 4, 6, 2, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 4]
Average loss at step 100: 0.000000 learning rate: 10.000000
Minibatch perplexity: 0.00
[6, 6, 6, 2, 2, 5, 5, 5, 3, 3, 1, 1, 1, 7, 4, 2, 2, 2, 6, 5, 3, 3, 3, 5, 6, 4, 4, 4, 4, 7]
Average loss at step 200: 0.000000 learning rate: 10.000000
Minibatch perplexity: 0.00
[6, 2, 6, 6, 2, 5, 3, 5, 5, 3, 4, 4, 1, 1, 4, 3, 5, 2, 2, 5, 2, 6, 3, 3, 6, 1, 7, 4, 4, 7]
Average loss at step 300: 0.000000 learning rate: 10.000000
Minibatch perplexity: 0.00
[6, 2, 6, 6, 6, 5, 3, 5, 5, 5, 1, 4, 1, 4, 1, 2, 5, 2, 3, 2, 3, 6, 3, 2, 3, 4, 7, 4, 1, 4]
Average loss at step 400: 0.000000 learning rate: 10.000000
Minibatch perplexity: 0.00
[6, 2, 2, 6, 2, 5, 3, 3, 5, 3, 1, 4, 4, 1, 7, 2, 5, 5, 2, 6, 3, 6, 6, 3, 5, 4, 7, 7, 4, 4]
Average loss at step 500: 0.000000 learning rate: 10.000000
Minibatch perplexity: 0.00
[2, 2, 6, 2, 

Exception AssertionError: AssertionError() in <bound method InteractiveSession.__del__ of <tensorflow.python.client.session.InteractiveSession object at 0x7ff2fbbc1f10>> ignored
Exception AssertionError: AssertionError() in <generator object get_controller at 0x7ff2fc5b6fa0> ignored


In [52]:
predict([1,2,3], 4, 0), predict([1,2,3], 4, 1)

([1, 2, 3, 4, 5, 6, 7], [1, 2, 3, 4, 5, 6, 7])

In [50]:
predict([7,6,5], 4, 0), predict([7,6,5], 4, 1)

([7, 6, 5, 4, 3, 4, 1], [7, 6, 5, 1, 2, 3, 4])

In [53]:
predict([1,2,3,7], 3, 0), predict([1,2,3,4], 3, 1)

([1, 2, 3, 7, 6, 5, 1], [1, 2, 3, 4, 5, 6, 7])

In [ ]:
batches = data_gen.next_batch(unrollings_adjuster=unrollings_adjuster, num_unrollings=n_unrollings+1)
# batches_encoded = data_gen.encode_batches(batches,n_unrollings+1)

In [ ]:
batches

In [ ]:
data_gen.encode_batches(batches,n_unrollings+1+2)

In [ ]:
data_gen.encode_batches(batches,n_unrollings+1)

In [ ]:
batches

In [ ]:
n_unrollings+1

In [ ]:
data_gen = BatchGenerator(2,4)
batches = data_gen.next_batch()
batches
# batches_encoded = data_gen.encode_batches(batches)

In [ ]:
# Creates a graph.
a = tf.constant([[1.0, 2.0, 3.0], [1.0, 2.0, 3.0]], shape=[2, 3], name='a')
b = tf.constant([[1.0, 2.0], [1.0, 2.0], [1.0, 2.0]], shape=[3, 2], name='b')
c = tf.matmul(a, b)


print a.eval()
print b.eval()
print c.eval()

In [ ]:
# Creates a graph.
a = tf.constant([[1.0, 2.0, 3.0], [1.0, 2.0, 3.0]], shape=[2, 3], name='a')
b = tf.constant([[1.0, 2.0], [1.0, 2.0], [1.0, 2.0]], shape=[3, 2], name='b')
d = tf.constant([[1.0, 2.], [3., 4.]], name='d')
c = tf.matmul(a, b)+d



print a.eval()
print b.eval()
print d.eval()
print c.eval()

In [ ]:
input = tf.constant([[[1, 1, 1], [2, 2, 2]],
                     [[3, 3, 3], [4, 4, 4]],
                     [[5, 5, 5], [6, 6, 6]]])
i1 = tf.slice(input, [1, 0, 0], [1, 1, 3])
# tf.slice(input, [1, 0, 0], [1, 2, 3]) ==> [[[3, 3, 3],
#                                             [4, 4, 4]]]
# tf.slice(input, [1, 0, 0], [2, 1, 3]) ==> [[[3, 3, 3]],
#                                            [[5, 5, 5]]]
print input, i1
print input.eval()
print
print i1.eval()

In [ ]:
input = tf.constant([[1, 1, 2],
                     [3, 3, 4],
                     [5, 5, 6]])
# i2 = tf.slice(input, [0, 0], [3, 2])
s = input.get_shape()
i2 = tf.slice(input, [0, 0], np.array(s.as_list())-np.array([0,2]))
# tf.slice(input, [1, 0, 0], [1, 2, 3]) ==> [[[3, 3, 3],
#                                             [4, 4, 4]]]
# tf.slice(input, [1, 0, 0], [2, 1, 3]) ==> [[[3, 3, 3]],
#                                            [[5, 5, 5]]]
print input, i2
print input.eval()
print
print i2.eval()

In [ ]:
s = input.get_shape()



In [ ]:


ini_steps = [2,3,1]
jrn_ids = map(data_gen.val2id, ini_steps)
jrn_1h = map(data_gen.encode_1h, jrn_ids)


# for _ in range(7-len(jrn_ids)):
for _ in range(10-len(jrn_ids)):
    sample_inputs, model = get_predictor(len(jrn_ids))

    dick ={}
    for i, s in zip(sample_inputs, jrn_1h):                        
        dick[i]=s

    prediction = model.eval(dick)

    pred_step_id = data_gen.weighted_pick(prediction[-1].ravel())                                          
    jrn_ids.append(pred_step_id)                 

    pred_step_1h = data_gen.encode_1h(pred_step_id)                    
    jrn_1h.append(pred_step_1h)                    



print map(lambda w: data_gen.id2val(w), jrn_ids)

In [ ]:
arr = [1,2,3]
print np.pad(arr,pad_width=3, mode='reflect')
print np.pad(arr,pad_width=3, mode='symmetric')
print np.pad(arr,pad_width=3, mode='wrap')
print np.pad(arr,pad_width=3, mode='edge')
print np.pad(arr,pad_width=3, mode='constant',constant_values=9)
print np.pad(arr,pad_width=3)

In [ ]:
graph = tf.Graph()
with graph.as_default():        

    # Input data.
    train_data = list()
    for _ in range(n_unrollings+1):
        train_data.append(
            tf.placeholder(tf.float32, shape=[batch_size, vocabulary_size]))
    train_inputs = train_data[:n_unrollings]
    train_labels = train_data[1:]  # labels are inputs shifted by one time step.
    train_labels_concat = tf.concat(0, train_labels)
    
    #social Input data
    social_input = tf.placeholder(tf.float32, shape=[batch_size, social_vocabulary_size])
    
    with tf.variable_scope("rnn") as rnn_scope:
    
        
    
        # RNN struct
        cell = rnn_cell.LSTMCell(num_units=n_hidden, input_size=vocabulary_size)     
        outputs, states = rnn.rnn(cell, train_inputs, dtype=tf.float32) 
        outputs_concat = tf.concat(0, outputs)#tf.reshape(tf.concat(1, outputs), (-1, n_hidden))

        # Classifier.
        W_hy = tf.get_variable("W_hy", [n_hidden, vocabulary_size])
        b_hy = tf.get_variable("b_hy", [vocabulary_size])
        
        
        W_social = tf.get_variable("W_social", [n_hidden, social_vocabulary_size])
        b_social = tf.get_variable("b_social", [social_vocabulary_size])
        
        logits = tf.matmul(outputs_concat, W_hy) + b_hy

        # Loss func
        loss = tf.reduce_mean(
                tf.nn.softmax_cross_entropy_with_logits(logits, train_labels_concat))

        
        # Optimizer.
        global_step = tf.Variable(0, name='global_step')
        learning_rate = tf.train.exponential_decay(10.0, global_step, 5000, 0.1, staircase=True)
        optimizer = tf.train.GradientDescentOptimizer(learning_rate)
        gradients, v = zip(*optimizer.compute_gradients(loss))
        gradients, _ = tf.clip_by_global_norm(gradients, 1.25)
        optimizer = optimizer.apply_gradients(
                zip(gradients, v), global_step=global_step)


        # Predictions.
        train_prediction = tf.nn.softmax(logits)
        
        #perplexity
#         predictions[predictions < 1e-10] = 1e-10
#         return np.sum(np.multiply(labels, -np.log(predictions))) / labels.shape[0]        
        perplexity = tf.reduce_sum(tf.mul(train_labels_concat, -tf.log(train_prediction)))/train_labels_concat.get_shape()[0].value
        
        loss_summary = tf.scalar_summary("loss", loss)
        learning_rate_summary = tf.scalar_summary("learning_rate", learning_rate)
        perplexity_summary = tf.scalar_summary("perplexity_summary", perplexity)
    
